<a href="https://colab.research.google.com/github/Ayon150/AI/blob/main/VGG_16_build_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available()  else 'cpu'

In [ ]:
batch_size = 32
learning_rate = 0.0001
epochs = 10

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((128, 128)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
train_data = datasets.CIFAR10('data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:13<00:00, 12.5MB/s]


In [ ]:
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
class VGG16_Own(nn.Module):
  def __init__(self, num_class):
    super().__init__()
    self.feature_1 = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)

    )
    self.feature_2 = nn.Sequential(
        nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)

    )
    self.feature_3 = nn.Sequential(
        nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)

    )
    self.feature_4 = nn.Sequential(
        nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)

    )
    self.feature_5 = nn.Sequential(
        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.AdaptiveAvgPool2d((7,7))

    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=512*7*7, out_features=4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(in_features=4096, out_features=4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(in_features=4096, out_features=num_class)
    )

  def forward(self, x):
    x = self.feature_1(x)
    x = self.feature_2(x)
    x = self.feature_3(x)
    x = self.feature_4(x)
    x = self.feature_5(x)
    x = self.classifier(x)

    return x

In [ ]:
model = VGG16_Own(10).to(device=device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params= model.parameters(), lr = learning_rate)

In [ ]:
train_loss = []
test_loss = []


for epoch in range(epochs):
  batch_loss = 0
  acc = 0
  correct = 0
  total = 0

  model.train()
  for X, y in tqdm(train_dataloader):
    X, y = X.to(device), y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    batch_loss += loss.item()

  batch_loss /= len(train_dataloader)
  train_loss.append(batch_loss)


  model.eval()
  batch_loss = 0
  for X,y in tqdm(test_dataloader):
    X, y = X.to(device), y.to(device)
    with torch.inference_mode():
      y_pred = model(X)
      loss = loss_fn(y_pred, y)
      batch_loss += loss.item()
      correct += (y == y_pred.argmax(1)).sum().item()
      total += y_pred.size(0)

  batch_loss /= len(test_dataloader)
  test_loss.append(batch_loss)
  acc = correct / total * 100


  print(f"Train loss: {train_loss[-1]: .2f} | Test loss: {test_loss[-1]: .2f} | Accuracy: {acc: .2f} ")

T



  0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

Train loss:  1.77 | Test loss:  1.44 | Accuracy:  46.70 


  0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

...

  0%|          | 0/313 [00:00<?, ?it/s]

Train loss:  0.26 | Test loss:  0.65 | Accuracy:  80.89 


  0%|          | 0/1563 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

Train loss:  0.20 | Test loss:  0.62 | Accuracy:  81.72 
